In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error

def calculate_mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero = y_true != 0
    return np.mean(np.abs((y_true[non_zero] - y_pred[non_zero]) / y_true[non_zero])) * 100

def plot_predictions(y_test, predictions, title):
    plt.scatter(y_test, predictions, alpha=0.3)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
    plt.title(title)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.show()

def train_evaluate_bpf(dataset, input_columns, output_column, epochs, learning_rate, train_val_split, shuffle=False):
    if shuffle:
        dataset = dataset.sample(frac=1).reset_index(drop=True)

    X = dataset[input_columns].values
    y = dataset[output_column].values.reshape(-1, 1)

    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Split the dataset
    split_index = int(len(X_scaled) * train_val_split)
    X_train_val, X_test = X_scaled[:split_index], X_scaled[split_index:]
    y_train_val, y_test = y[:split_index], y[split_index:]

    # Define the BP_F network architecture dynamically based on input columns
    num_units = [len(input_columns), 10, 5, 1]
    model = Sequential()
    model.add(Dense(units=num_units[0], input_dim=len(input_columns), activation='relu'))
    for units in num_units[1:]:
        model.add(Dense(units=units, activation='relu' if units > 1 else 'linear'))  # Use linear activation for output layer

    # Compile the model with Adam optimizer and MSE loss function
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')

    # Train the BP_F network
    model.fit(X_train_val, y_train_val, epochs=epochs, verbose=1)

    # Evaluate the network
    predictions = model.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)
    mape = calculate_mape(y_test, predictions)

    # Visualize predictions
    plot_predictions(y_test, predictions, f'Real vs Predicted Values')
    
    return mae, mape

# Load the normalized datasets and drop the last column from dataset1
dataset1 = pd.read_csv('normalized_dataset1.csv')
dataset2 = pd.read_csv('normalized_dataset2.csv')
dataset3 = pd.read_csv('normalized_dataset3.csv')

dataset1 = dataset1.drop(columns=[dataset1.columns[-1]])


# Specify the parameters for each dataset
architecture_params = {
    'dataset1': {'dataset': dataset1, 'input_columns': dataset1.columns[:-1], 'output_column': dataset1.columns[-1], 'epochs': 500, 'learning_rate': 0.002, 'train_val_split': 0.85, 'shuffle': False},
    'dataset2': {'dataset': dataset2, 'input_columns': dataset2.columns[:-1], 'output_column': dataset2.columns[-1], 'epochs': 500, 'learning_rate': 0.002, 'train_val_split': 0.80, 'shuffle': False},
    'dataset3': {'dataset': dataset3, 'input_columns': dataset3.columns[:-1], 'output_column': dataset3.columns[-1], 'epochs': 500, 'learning_rate': 0.002, 'train_val_split': 0.80, 'shuffle': True}  # Assuming shuffling is needed for dataset3
}

# Train and evaluate BP_F network for each dataset
for name, params in architecture_params.items():
    print(f"Training and evaluating {name}...")
    mae, mape = train_evaluate_bpf(**params)
    print(f"{name} - MAE: {mae:.4f}, MAPE: {mape:.2f}%\n")


FileNotFoundError: [Errno 2] No such file or directory: 'normalized_cleaned_dataset1.csv'